In [169]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd

def load_data(ticker):

    filename = '../data/' + ticker + '_1min_firstratedata.csv'
    df = pd.read_csv(filename)
    df['ticker'] = ticker

    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["date"] = df["timestamp"].dt.date
    df["time"] = df["timestamp"].dt.time
    return df

ticker = 'SPY'
df = load_data(ticker)

In [170]:
# Add the RSI indicator with period n to a df

def RSI(df, n):
    "function to calculate RSI"
    delta = df["close"].diff()
    delta = delta[1:]
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    df["up"] = up.round(4)
    df["down"] = down.round(4)
    AVG_Gain = df["up"].rolling(window=n).mean()
    AVG_Loss = abs(df["down"].rolling(window=n).mean())
    RS = AVG_Gain / AVG_Loss
    RSI = 100.0 - (100.0 / (1.0 + RS))
    df["RSI_14"] = RSI.round(4)
    df = df.drop(columns=["up", "down"])

    return df

df = RSI(df, 14)

#df.head()

In [171]:
import datetime
import plotly.graph_objects as go


# Candlestick chart of one day
def chart_range(df, date):
    df = df[df["date"] == date]
    fig = go.Figure(
        data=[
            go.Candlestick(
                x=df["timestamp"],
                open=df["open"],
                high=df["high"],
                low=df["low"],
                close=df["close"],
            )
        ]
    )
    fig.show()


# chart_range(df, datetime.date(2023, 5, 18))


# chart the RSI of one day
def chart_rsi(df, date):
    df = df[df["date"] == date]
    # only chart from 9:30 to 16:00
    df = df[df["time"] >= datetime.time(9, 30, 0)]
    df = df[df["time"] <= datetime.time(16, 0, 0)]
    fig = go.Figure(data=go.Scatter(x=df["timestamp"], y=df["RSI_14"]))
    # add horizontal line at 30 and 70
    fig.add_shape(
        type="line",
        x0=df["timestamp"].min(),
        y0=30,
        x1=df["timestamp"].max(),
        y1=30,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )
    fig.add_shape(
        type="line",
        x0=df["timestamp"].min(),
        y0=70,
        x1=df["timestamp"].max(),
        y1=70,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )
    fig.show()


# chart_rsi(df, datetime.date(2023, 5, 18))


def chart_rsi_with_trades(timestamps, rsi_values, enters, exits):
    fig = go.Figure(
        data=go.Scatter(x=timestamps, y=rsi_values, mode="lines", name="RSI")
    )
    fig.add_trace(
        go.Scatter(
            x=enters["timestamps"],
            y=enters["values"],
            mode="markers",
            marker=dict(color="green", size=8),
            name="Enter",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=exits["timestamps"],
            y=exits["values"],
            mode="markers",
            marker=dict(color="red", size=8),
            name="Exit",
        )
    )
    # Add horizontal lines at RSI 30 and 70
    fig.add_shape(
        type="line",
        x0=timestamps[0],
        y0=30,
        x1=timestamps[-1],
        y1=30,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )
    fig.add_shape(
        type="line",
        x0=timestamps[0],
        y0=70,
        x1=timestamps[-1],
        y1=70,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )

    fig.show()

In [172]:
import datetime
import pandas as pd

def simulate_trade_RSI(date, entry, exit, max_trades, df):
    df_day = df[(df["date"] == date) & (df["time"] >= datetime.time(9, 30, 0)) & (df["time"] <= datetime.time(16, 0, 0))]
    trade_log = pd.DataFrame(columns=["Date", "Direction", "Entry", "Exit", "TimeEnter", "TimeExit", "Gain%", "Gain$"])
    trades, in_trade = 0, False

    for i in range(1, len(df_day)):
        rsi_condition = (df_day["RSI_14"].iloc[i - 1] < entry and df_day["RSI_14"].iloc[i] >= entry)
        exit_condition = (df_day["RSI_14"].iloc[i - 1] > exit and df_day["RSI_14"].iloc[i] <= exit)
        
        if not in_trade and rsi_condition:
            entry_price, in_trade, time_enter = df_day["close"].iloc[i], True, df_day["time"].iloc[i - 1]

        elif in_trade and (exit_condition or i == len(df_day) - 1):
            exit_price = df_day["close"].iloc[i]
            trade_gain_percent = round(((exit_price - entry_price) / entry_price) * 100, 2)
            trade_gain_net = round(exit_price - entry_price, 2)

            trade_log.loc[len(trade_log)] = pd.Series({
                "Date": date,
                "Direction": "Long",
                "Entry": entry_price,
                "Exit": exit_price,
                "TimeEnter": time_enter,
                "TimeExit": df_day["time"].iloc[i],
                "Gain%": trade_gain_percent,
                "Gain$": trade_gain_net,
            })

            trades, entry_price, in_trade = trades + 1, 0, False

            # Check if the maximum number of trades is reached
            if trades >= max_trades:
                break

    return trade_log

In [173]:
# Combine the simulated trades for a range of dates into a single dataframe

def simulate_date_range(start_date, end_date, entry, exit, max_trades, df):
    day_datas = []

    for n in range((end_date - start_date).days + 1):
        date = start_date + datetime.timedelta(n)
        day_data = simulate_trade_RSI(date, entry, exit, max_trades, df)
        if day_data is not None:
            day_datas.append(day_data)

    day_datas = pd.concat(day_datas, ignore_index=True)

    return day_datas

In [174]:
#trades = simulate_date_range(datetime.date(2022, 9, 30), datetime.date(2023, 9, 30), 30, 70, df)

# post-processing of the trades dataframe

def rsi_backtest_explain_results(trades):

    total_gain = trades["Gain$"].sum()
    total_trades = len(trades)

    average_percent_gain = round(trades["Gain%"].mean(), 2)

    print("Total trades: " + str(total_trades))
    print("Average percent gain per trade: " + str(average_percent_gain) + "%")
    print("Total percent gain: " + str(round(total_gain, 2)) + "%")

    # histogram of the percent gain per trade
    import plotly.express as px

    fig = px.histogram(trades, x="Gain%", nbins=100)
    fig.show()

def rsi_ranged_backtest_single_pnl(df, rsi_enter, rsi_exit):
    res = pd.DataFrame()
    res.loc[0, 'StartDate'] = df['Date'].min()
    res.loc[0, 'EndDate'] = df['Date'].max()
    res.loc[0, 'Entry'] = rsi_enter
    res.loc[0, 'Exit'] = rsi_exit
    res.loc[0, 'TotalTrades'] = len(df)
    res.loc[0, 'TotalPercentGain'] = round(df['Gain%'].sum(), 2)
    res.loc[0, 'AveragePercentGain'] = round(df['Gain%'].mean(), 2)
    res.loc[0, 'MaxPercentGain'] = round(df['Gain%'].max(), 2)
    res.loc[0, 'MinPercentGain'] = round(df['Gain%'].min(), 2)

    return res

In [175]:
# Full stack experiment

ticker = 'SPY'
max_trades = 2

df = load_data(ticker)
df = RSI(df, 14)

enter_exit_pairs_symetric = [(30, 70), (20, 80)]#, (25, 75), (35, 65), (40, 60)]
enter_exit_pairs_asymetric = [(10, 30), (20, 40)]#, (30, 50), (40, 60), (50, 70), (60, 80), (70, 90)]

all_pairs = enter_exit_pairs_symetric + enter_exit_pairs_asymetric

results = pd.DataFrame(columns=['StartDate', 'EndDate', 'Entry', 'Exit', 'TotalTrades', 'TotalPercentGain', 'AveragePercentGain', 'MaxPercentGain', 'MinPercentGain'])
for e in all_pairs:
    trades = simulate_date_range(datetime.date(2022, 9, 30), datetime.date(2023, 9, 30), e[0], e[1], max_trades, df)
    pnl = rsi_ranged_backtest_single_pnl(trades, e[0], e[1])
    results.loc[len(results)] = pnl.loc[0]

In [176]:
results

,StartDate,EndDate,Entry,Exit,TotalTrades,TotalPercentGain,AveragePercentGain,MaxPercentGain,MinPercentGain
0,2022-09-30,2023-09-29,30.0,70.0,500.0,-1.76,-0.0,1.04,-1.7
1,2022-09-30,2023-09-29,20.0,80.0,403.0,1.32,0.0,1.32,-1.94
2,2022-09-30,2023-09-29,10.0,30.0,135.0,0.02,0.0,2.13,-1.13
3,2022-09-30,2023-09-29,20.0,40.0,434.0,8.15,0.02,1.27,-0.88
